In [ ]:
import sys
import gc
import os, shutil
import tempfile
from os import listdir
import clr_callback

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
import seaborn as sns
from PIL import Image

from keras import backend as K

import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image
from tensorflow.keras import mixed_precision, regularizers
from tensorflow.keras.metrics import top_k_categorical_accuracy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

from sklearn.metrics import classification_report,confusion_matrix, matthews_corrcoef
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
plt.set_cmap('gray')
pd.set_option('precision', 3)

In [ ]:
for gpu in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

# Utility Functions

In [ ]:
def create_datagen(train_path, val_path, test_path, target_size = (256,256), batch_size = 16, efficient = False):
    if efficient:
        train_datagen = image.ImageDataGenerator(
            rescale = 1.,
        )
    else:
        train_datagen = image.ImageDataGenerator(
            rescale = 1./255,
        )

    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=target_size,
        batch_size= batch_size,
        color_mode="rgb",
        class_mode='categorical',
        shuffle = True
    )

    validation_generator = train_datagen.flow_from_directory(
        val_path,
        target_size=target_size,
        batch_size= batch_size,
        color_mode="rgb",
        class_mode='categorical',
        shuffle = False
    )
    
    test_generator = train_datagen.flow_from_directory(
        test_path,
        target_size=target_size,
        batch_size= batch_size,
        color_mode="rgb",
        class_mode='categorical',
        shuffle = False
    )
    return train_generator, validation_generator, test_generator

In [ ]:
plt.rcParams["figure.figsize"] = (9,12)

def plot_model_history(model, path):
    fig, (ax1, ax2) = plt.subplots(2)
    ax1.plot(model.history['accuracy'])
    ax1.plot(model.history['val_accuracy'])
    ax1.set_title('Model Accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train', 'Val'], loc='upper left')
    
    ax2.plot(model.history['loss'], 'b')
    ax2.plot(model.history['val_loss'], 'r')
    ax2.set_title('Training and Validation loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train Loss', 'Val Loss'], loc='upper left')
    fig.savefig(path, bbox_inches = 'tight')

    plt.show()

In [ ]:
def plot_confusion_matrix(df_confusion, path, title='Confusion Matrix'):
    print(df_confusion)
    plt.figure(figsize=(8, 6))
    plt.title(title)

    heatmaps = sns.heatmap(df_confusion, annot=True, cmap = "viridis",
               vmin = 0, vmax = 1)
    plt.setp(heatmaps.get_xticklabels(), rotation=30)
    plt.setp(heatmaps.get_yticklabels(), rotation=30)
    plt.tight_layout()
    plt.savefig(path)
    plt.show()

In [ ]:
def report_classifier(model,generator, testing_size, batch_size, evaluate = False, efficient_net = False, 
                      path_report = None):
    Y_pred = model.predict(generator)
    y_pred = np.argmax(Y_pred, axis=1)
    k = matthews_corrcoef(generator.classes, y_pred)
    print(f'Matthew Correlation Coefficient : {k:.2f}')
    
    df_confusion_ori = pd.crosstab(generator.classes, y_pred, 
                               rownames=['Actual'], colnames=['Predicted'], margins=True)
    df_confusion = pd.crosstab(generator.classes, y_pred, 
                               rownames=['Actual'], colnames=['Predicted'], margins=True, normalize = "index")
    df_confusion.rename(columns={0: 'Confined Masonry', 1 : 'RC Infilled', 2 : 'Timber', 3 : 'Unconfined'}, 
              index={0: 'Confined Masonry', 1 : 'RC Infilled', 2 : 'Timber', 3 : 'Unconfined'}, inplace = True)
    plot_confusion_matrix(df_confusion[:4][:], path = path_report)
    
    print('Classification Report')
    target_names = ['Confined', 'RC', 'Timber', 'Unconfined']
    print(classification_report(generator.classes, y_pred, target_names=target_names))
    
    if evaluate:
        tipologi = {0 : 'Confined', 1 : 'RC Infilled', 2 : 'Timber', 3 : 'Unconfined'}
        generator.reset()
        print_index = 0
        showimg = 1
        plt.figure(figsize=(16,8))
        while(print_index < len(y_pred)):
            x_batch, y_batch = next(generator)
            for k, (img, lbl) in enumerate(zip(x_batch, y_batch)):
                if(showimg == 4):
                    plt.figure(figsize=(16,8))
                    showimg = 1
                if (y_pred[print_index] != np.argmax(lbl)):
                    if efficient_net:
                        plt.subplot(1, 4, showimg)#4 rows with 8 images.
                        showimg += 1
                        plt.title('Prediksi :' + str(tipologi[y_pred[print_index]]) + ', Aktual :' + str(tipologi[np.argmax(lbl)]), 
                                  fontsize = 9)
                        plt.axis('off')
                        plt.tight_layout()
                        plt.imshow(img/255.)
                    else:
                        plt.subplot(1, 4, showimg)#4 rows with 8 images.
                        showimg += 1
                        plt.title('Prediksi :' + str(y_pred[print_index]) + ', Aktual :' + str(np.argmax(lbl)), 
                                  fontsize = 9)
                        plt.axis('off')
                        plt.tight_layout()
                        plt.imshow(img)
                print_index += 1
    return

# Domain Transfer Test

In [ ]:
model = tf.keras.models.load_model('Deep Learning Models/Typology Classifier/Model RSL - Mobile - D2 - 16.h5')

In [ ]:
train_path = 'train_path'
val_path = 'val_path'
test_path = 'test_path'

In [ ]:
train_generator, validation_generator, test_generator = create_datagen(train_path, val_path, test_path, 
                                                                       batch_size = 16, efficient = True)

In [ ]:
report_classifier(model,test_generator,202,16,evaluate = True, efficient_net = True, 
                 path_report = 'Misc/temp Graph/'+'DT Model RS-L to HP-L - D3 - '+' Report.jpg')